# IR Lab SoSe 2024: Baseline Retrieval System

This jupyter notebook serves as baseline retrieval system that you can try to improve upon.
We will use the a corpus of scientific papers (title + abstracts) from the fields of information retrieval and natural language processing (the [IR Anthology](https://ir.webis.de/anthology/) and the [ACL Anthology](https://aclanthology.org/)). This serves Jupyter notebook only serves as retrieval system, i.e., it gets a set of information needs (topics) and a corpus as input and produces a run file as output. Please do evaluations in a new dedicated notebook.

### Step 1: Import Libraries

We will use [tira](https://www.tira.io/), an information retrieval shared task platform, for loading the (pre-built) retrieval index and [ir_dataset](https://ir-datasets.com/) to subsequently build a retrieval system with [PyTerrier](https://github.com/terrier-org/pyterrier), an open-source search engine.

Building your own index can be already one way that you can try to improve upon this baseline (if you want to focus on creating good document representations). Other ways could include reformulating queries or tuning parameters or building better retrieval pipelines.

In [1]:
# You only need to execute this cell if you are using Google Golab.
# If you use GitHub Codespaces, everything is already installed.
!pip3 install tira ir-datasets python-terrier

In [2]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pandas as pd
import pyterrier as pt

In [3]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


### Step 2: Load the Dataset and the Index

The type of the index object that we load is `<class 'jnius.reflect.org.terrier.structures.Index'>`, in fact a [Java class](http://terrier.org/docs/v3.6/javadoc/org/terrier/structures/Index.html) wrapped into Python. However, you do not need to worry about this: at this point, we will simply use the provided Index object to run procedures defined in Python.

In [67]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

# A (pre-built) PyTerrier index loaded from TIRA
indexNonstem = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

# Index created with PorterStemmer
indexer = pt.IterDictIndexer("./tmp/index", overwrite=True, stemmer=None)
index_ref = indexer.index(pt_dataset.get_corpus_iter())
indexStem = pt.IndexFactory.of(index_ref)

ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:  24%|██▍       | 30834/126958 [00:04<00:17, 5573.52it/s]Exception in thread Thread-83 (_write_fifos):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyterrier/index.py", line 1093, in _write_fifos
    json.dump(doc, fifo)
  File "/usr/lib/python3.10/json/__init__.py", line 180, in dump
    fp.write(chunk)
BrokenPipeError: [Errno 32] Broken pipe
ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:   0%|          | 104/126958 [37:48<768:32:16, 21.81s/it]
ir-lab-sose-2024/ir-acl-anthology-20240504-training documents:  71%|███████   | 90006/126958 [00:13<00:05, 6684.01it/s]

16:20:36.925 [ForkJoinPool-13-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (2020.mir_conference-2020.1) - further warnings are suppressed


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:17<00:00, 7330.83it/s] 


16:20:43.730 [ForkJoinPool-13-worker-3] WARN org.terrier.structures.indexing.Indexer - Indexed 3 empty documents


### Step 3: Define the Retrieval Pipeline

We will define a BM25 retrieval pipeline as baseline. For details, see:

- [https://pyterrier.readthedocs.io](https://pyterrier.readthedocs.io)
- [https://github.com/terrier-org/ecir2021tutorial](https://github.com/terrier-org/ecir2021tutorial)

In [68]:
bm25_stemmed = pt.BatchRetrieve(indexStem, wmodel="BM25")
bm25_nonstemmed = pt.BatchRetrieve(indexNonstem, wmodel="BM25")

### Step 4: Create the Run


In [54]:
print('First, we have a short look at the first three topics:')

pt_dataset.get_topics('text').head(3)

First, we have a short look at the first three topics:


,qid,query
0,1,retrieval system improving effectiveness
1,2,machine learning language identification
2,3,social media detect self harm


In [55]:
#print('Now we do the retrieval...')
run = bm25_stemmed(pt_dataset.get_topics('text'))
run.head(5)

,qid,docid,docno,rank,score,query
0,1,94858,2004.cikm_conferenc,0,15.681777,retrieval system improving effectiveness
1,1,125137,1989.ipm_journal-ir,1,15.047380,retrieval system improving effectiveness
2,1,125817,2005.ipm_journal-ir,2,14.144223,retrieval system improving effectiveness
3,1,5868,W05-0704,3,14.025748,retrieval system improving effectiveness
4,1,84876,2016.ntcir_conferen,4,13.947994,retrieval system improving effectiveness


In [70]:
#Not Stemmed
#print('Now we do the retrieval...')
run = bm25_stemmed(pt_dataset.get_topics('text'))

### Step 5: Persist the run file for subsequent evaluations

The output of a prototypical retrieval system is a run file. This run file can later (optimally in a different notebook) be statistically evaluated.

In [71]:
persist_and_normalize_run(run, system_name='bm25-baseline', default_output='../runs')

The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt".
